# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [100]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import time

# Import API key
from api_keys import geoapify_key

In [101]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file_path = Path("output_data/cities.csv")
cities_df = pd.read_csv(csv_file_path)


# Display sample data
cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ljusdal,61.8288,16.0913,274.82,96,75,0.83,SE,1731218553
1,1,Muroto-misakicho,33.2833,134.1500,293.15,71,100,2.86,JP,1731218555
2,2,Port elizabeth,-33.9180,25.5701,292.21,80,20,1.54,ZA,1731218080
3,3,Saipan,15.1355,145.7010,301.84,83,51,6.26,MP,1731218560
4,4,Hawaiian paradise park,19.5933,-154.9731,295.78,91,26,0.45,US,1731218562


In [102]:
print(type(cities_df))


<class 'pandas.core.frame.DataFrame'>


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [103]:
%%capture --no-display

# Configure the map plot
map_plot = cities_df.hvplot.points(
    'Lng',
    'Lat',
    size = 'Humidity',
    color = 'Humidity',
    title = 'Map of Humidity of Cities',
    geo = True,
    tiles = 'OSM'
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [104]:
# Narrow down cities that fit criteria and drop any results with null values
cities_df1 = cities_df[cities_df['Max Temp'] >= 292]
cities_df2 = cities_df1[cities_df1['Wind Speed'] <= 15]

# Drop any rows with null values
cities_df3 = cities_df2.dropna()

# Display sample data
print(cities_df3)

     City_ID                    City      Lat       Lng  Max Temp  Humidity  \
1          1        Muroto-misakicho  33.2833  134.1500    293.15        71   
2          2          Port elizabeth -33.9180   25.5701    292.21        80   
3          3                  Saipan  15.1355  145.7010    301.84        83   
4          4  Hawaiian paradise park  19.5933 -154.9731    295.78        91   
6          6                Mabaruma   8.2000  -59.7833    296.00       100   
..       ...                     ...      ...       ...       ...       ...   
588      588             Kailua-kona  19.6406 -155.9956    297.35        90   
590      590           Talata mafara  12.5732    6.0597    292.03        29   
592      592                 Ha tinh  18.3333  105.9000    300.04        56   
594      594           Haiku-pauwela  20.9219 -156.3051    297.06        79   
596      596              Carutapera  -1.1950  -46.0200    298.80        84   

     Cloudiness  Wind Speed Country        Date  
1

In [105]:
print(type(cities_df3))

<class 'pandas.core.frame.DataFrame'>


### Step 3: Create a new DataFrame called `hotel_df`.

In [106]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(cities_df3.copy())

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel_Name'] = None

# Display sample data
print(hotel_df)

     City_ID                    City      Lat       Lng  Max Temp  Humidity  \
1          1        Muroto-misakicho  33.2833  134.1500    293.15        71   
2          2          Port elizabeth -33.9180   25.5701    292.21        80   
3          3                  Saipan  15.1355  145.7010    301.84        83   
4          4  Hawaiian paradise park  19.5933 -154.9731    295.78        91   
6          6                Mabaruma   8.2000  -59.7833    296.00       100   
..       ...                     ...      ...       ...       ...       ...   
588      588             Kailua-kona  19.6406 -155.9956    297.35        90   
590      590           Talata mafara  12.5732    6.0597    292.03        29   
592      592                 Ha tinh  18.3333  105.9000    300.04        56   
594      594           Haiku-pauwela  20.9219 -156.3051    297.06        79   
596      596              Carutapera  -1.1950  -46.0200    298.80        84   

     Cloudiness  Wind Speed Country        Date Hot

In [107]:
print(type(hotel_df))

<class 'pandas.core.frame.DataFrame'>


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [109]:
# Set parameters to search for a hotel; used ChatGPT for help on code
limit = 15
radius = 1000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame for the current city
    latitude = row['Lat']
    longitude = row['Lng']

    # Set the parameters for the search, using the current city's coordinates
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"

    params = {
        "filter": filters,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    # Set the base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Make the API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)
        
        # Check if the response status code is OK (200)
        if response.status_code == 200:
            # Convert the API response to JSON format
            name_address = response.json()

            # Grab the first hotel from the results and store the name in the hotel_df DataFrame
            try:
                hotel_df.loc[index, "Hotel_Name"] = name_address["features"][0]["properties"]["name"]
            except (KeyError, IndexError):
                # If no hotel is found, set the hotel name as "No hotel found".
                hotel_df.loc[index, "Hotel_Name"] = "No hotel found"
        else:
            print(f"Error: {response.status_code} - Unable to fetch data for {row['City']}")
            hotel_df.loc[index, "Hotel_Name"] = "Error fetching data"
    except requests.exceptions.RequestException as e:
        print(f"Request error for {row['City']}: {e}")
        hotel_df.loc[index, "Hotel_Name"] = "Request error"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel_Name']}")

    # Sleep for 3 seconds to avoid overloading the server
    time.sleep(3)

# Display the updated DataFrame with hotel names
print(hotel_df)

Starting hotel search
Error: 400 - Unable to fetch data for Muroto-misakicho
Muroto-misakicho - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Port elizabeth
Port elizabeth - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Saipan
Saipan - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Hawaiian paradise park
Hawaiian paradise park - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Mabaruma
Mabaruma - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Borba
Borba - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Port mathurin
Port mathurin - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Lazaro cardenas
Lazaro cardenas - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Adamstown
Adamstown - nearest hotel: Error fetching data
Error: 400 - Unable to fetch data for Port alfred
Port alfred - neare

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [110]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    size = 'Humidity',
    color = 'Humidity',
    title = 'Map of Hotels & Humidity',
    geo = True,
    tiles = 'OSM'
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)